# pytorch 基础

本课内容包括
1. 基本的pytorch入门
2. pytorch中向量、导数、偏导操作，参考之前线性代数以及微积分
3. 基于pytorch构建一个线性回归网络
4. 基于pytorch实现class1 对图片的分类以及优化

---

参考资料

1. [ws university pytorch introudction](https://courses.cs.washington.edu/courses/cse446/19au/section9.html)
2. [li hong yi](https://www.youtube.com/watch?v=kQeezFrNoOg)
3. [pytorch homepage](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

## 1.安装pytorch

与tensorflow相同，pytorch分为cpu版本与gpu版本，官网有相对于安装代码
[安装页面](https://pytorch.org/)



参看本地计算机是否支持GPU以及GPU型号

In [10]:
import torch
def pytrochGPU(memoryview=False):
    support=torch.cuda.is_available()
    if support==True:
        print("GPU is available")
        print("GPU的数量",torch.cuda.device_count())
        print("GPU的名字",torch.cuda.get_device_name(0))
        print("当前GPU索引",torch.cuda.current_device())
if __name__ == '__main__':
    print(pytrochGPU())


GPU is available
GPU的数量 1
GPU的名字 NVIDIA GeForce RTX 3090
当前GPU索引 0
None


由于可能本地计算机没有gpu，为了避免出错，首先判断是GPU还是CPU，并将数据在对应设备上进行运算

In [14]:
import numpy as np
device="cuda" if torch.cuda.is_available() else "cpu"
x=torch.randn(2,3)
x=x.to(device)
print(x.device)


cuda:0



pytorch 中torch的作用类似numpy，包括创建数组，进行线性代数运算等，但torch还可以直接进行求导运算

numpy 与 torch之间也可以相互转化


见下代码


In [17]:
x_tensor=torch.tensor([2,3,2])
print(type(x_tensor))
print(x_tensor)

<class 'torch.Tensor'>
tensor([2, 3, 2])
